In [3]:
import pprint
import sys
import json
import spotipy
import spotipy.util as util
import pprint
import pandas as pd
from pandas.io.json import json_normalize
import math
import time
import numpy as np
sleep_min = 1
sleep_max = 5

In [17]:
# load dataset from previous notebook
df = pd.read_csv('../datasets/nerken_user_songs.csv')

In [19]:
# display all columns without jupyter notebook limitation

pd.options.display.max_columns = None
display(df)

,Unnamed: 0,Id,Name,Playlist_Id,Album_Id,Date_added,Duration,Popularity,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Key,Mode,Track_Number
0,0,3wD62ofDBqgeJSW2u7WKil,Ragtime Cat (feat. Lilja Bloom),3ypuzkCtKGOQ0Y63x295Jx,5VOdrHq3pi0FTHQZtBaj7V,2018-10-24T14:52:55Z,181000.0,21.0,0.736,0.715,-4.114,0.0412,0.012000,0.000796,0.2180,0.8510,119.976,5,0,1.0
1,1,14EeHJP2ZhOqabZgCAF27r,Artichaut,3ypuzkCtKGOQ0Y63x295Jx,5VOdrHq3pi0FTHQZtBaj7V,2018-10-24T14:52:55Z,265093.0,0.0,0.695,0.879,-5.655,0.1600,0.223000,0.197000,0.6890,0.9100,203.886,4,1,2.0
2,2,4iGiK3PUx1r9G6N3vkQERe,Extra Mile,3ypuzkCtKGOQ0Y63x295Jx,5VOdrHq3pi0FTHQZtBaj7V,2018-10-24T14:52:55Z,305280.0,0.0,0.690,0.722,-5.898,0.2610,0.268000,0.000529,0.9350,0.6610,175.889,0,1,3.0
3,3,2KTHFxwHv1eUUALTx7uBei,AAA (Triple A),3ypuzkCtKGOQ0Y63x295Jx,5VOdrHq3pi0FTHQZtBaj7V,2018-10-24T14:52:55Z,204026.0,15.0,0.671,0.806,-5.632,0.0777,0.056400,0.000003,0.7390,0.8690,129.980,7,1,4.0
4,4,2I13jeL6QfdArAhpqUzFSr,The Sound of Swing (Onur Engin Edit) [feat. Al...,3ypuzkCtKGOQ0Y63x295Jx,5VOdrHq3pi0FTHQZtBaj7V,2018-10-24T14:52:55Z,341493.0,13.0,0.544,0.815,-4.554,0.0772,0.008510,0.000017,0.1690,0.7440,90.015,7,1,5.0
5,5,2sm8jAsyuyoX45ytlnyBLj,No Diggity (Minimatic Remix),3ypuzkCtKGOQ0Y63x295Jx,5VOdrHq3pi0FTHQZtBaj7V,2018-10-24T14:52:55Z,212320.0,19.0,0.855,0.698,-6.697,0.0600,0.056300,0.063800,0.0750,0.6810,116.043,11,0,6.0
6,6,0x9zhyDsfltieIXYUv1XRg,Gangsterlove (Soundnomaden Remix),3ypuzkCtKGOQ0Y63x295Jx,5VOdrHq3pi0FTHQZtBaj7V,2018-10-24T14:52:55Z,253733.0,0.0,0.798,0.794,-5.287,0.0711,0.257000,0.000000,0.0872,0.7720,123.987,5,0,7.0
7,7,6cM9Wxrwik0TipzJPcZLG7,Swing You Winners (feat. Tape Five),3ypuzkCtKGOQ0Y63x295Jx,5VOdrHq3pi0FTHQZtBaj7V,2018-10-24T14:52:55Z,164080.0,12.0,0.669,0.771,-5.286,0.1910,0.008970,0.000061,0.2750,0.5920,169.916,7,1,8.0
8,8,4mpyBYJzpieMYwvtnnlzlY,Shadrack (Fonograff Rework),3ypuzkCtKGOQ0Y63x295Jx,5VOdrHq3pi0FTHQZtBaj7V,2018-10-24T14:52:55Z,228693.0,20.0,0.880,0.870,-5.851,0.3200,0.279000,0.000806,0.1160,0.7430,115.056,5,0,9.0
9,9,1Z0UlpDHhTmRRF2p5aZPa7,Vive le swing (Gabin Brassy Remix),3ypuzkCtKGOQ0Y63x295Jx,5VOdrHq3pi0FTHQZtBaj7V,2018-10-24T14:52:55Z,266133.0,0.0,0.786,0.831,-5.620,0.1200,0.072400,0.000000,0.8530,0.7990,117.964,10,0,10.0


In [21]:
username = 'nerken' # 'incautiouswifi' #some users don't have a name, just an ID
scope = 'user-follow-read,user-follow-modify,user-read-recently-played,user-top-read,user-library-read,user-library-modify,user-read-playback-state,user-read-currently-playing,user-modify-playback-state,playlist-read-collaborative,playlist-modify-private,playlist-modify-public,playlist-read-private,app-remote-control,user-read-email,user-read-private'

client_id = '22e782a724264812b124038bbf783396'
client_secret = '28bef740bacb435b9e4a8c0813183153'
oauth2 = util.oauth2.SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
token = util.prompt_for_user_token(username, scope=scope, client_id='22e782a724264812b124038bbf783396', client_secret='28bef740bacb435b9e4a8c0813183153', redirect_uri='https://localhost:8080')

In [6]:
# open a Spotify connection
sp = spotipy.Spotify(auth=token)

In [7]:
#for x in range(len(pd)):
def get_track_features(df):
    '''
    request audio features for each song to new dataframe 'track_features'
    '''
    track_features = pd.DataFrame()
    for x in range(len(df)):
        track_id = df['track.id'][x]
        if x%10 == 0:
            time.sleep(np.random.uniform(sleep_min, sleep_max))
        try:
            track_data = sp.audio_features(track_id)
        except:
            oauth2 = util.oauth2.SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
            token = util.prompt_for_user_token(username, scope=scope, client_id='22e782a724264812b124038bbf783396', client_secret='6219554f45764f2cbfe13306ec476f10', redirect_uri='https://localhost:8080')
            sp = spotipy.Spotify(auth=token)
            track_data = sp.audio_features(track_id)
        data = (json_normalize(track_data))
        track_features = track_features.append(data,ignore_index=True)
        track_features.to_csv(username+'_track_features.csv') # save all song_features to a file
        print(f'{track_id}')
    return track_features

In [1]:
# tracks_features = get_track_features(df) // This line returns the features for all songs - deactivated to avoid overwritting

In [39]:
# load saved tracks_features
tracks_features = pd.read_csv('../datasets/'+username+'_user_songs.csv')
tracks_features = tracks_features.drop(columns=['Unnamed: 0'],axis=1)

In [40]:
tracks_features.head(3)

,Id,Name,Playlist_Id,Album_Id,Date_added,Duration,Popularity,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Key,Mode,Track_Number
0,3wD62ofDBqgeJSW2u7WKil,Ragtime Cat (feat. Lilja Bloom),3ypuzkCtKGOQ0Y63x295Jx,5VOdrHq3pi0FTHQZtBaj7V,2018-10-24T14:52:55Z,181000.0,21.0,0.736,0.715,-4.114,0.0412,0.012,0.000796,0.218,0.851,119.976,5,0,1.0
1,14EeHJP2ZhOqabZgCAF27r,Artichaut,3ypuzkCtKGOQ0Y63x295Jx,5VOdrHq3pi0FTHQZtBaj7V,2018-10-24T14:52:55Z,265093.0,0.0,0.695,0.879,-5.655,0.1600,0.223,0.197000,0.689,0.910,203.886,4,1,2.0
2,4iGiK3PUx1r9G6N3vkQERe,Extra Mile,3ypuzkCtKGOQ0Y63x295Jx,5VOdrHq3pi0FTHQZtBaj7V,2018-10-24T14:52:55Z,305280.0,0.0,0.690,0.722,-5.898,0.2610,0.268,0.000529,0.935,0.661,175.889,0,1,3.0


In [ ]:
# some missing code when joining user songs from 01_file and adding the features.

In [ ]:
# Next file = 03_K-Means